In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-embedding-CNN/"

In [ ]:
import pickle
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-base/train_set.csv")
print (df.shape)
df_test = pd.read_csv('/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-base/test_set.csv')
df_test = df_test.dropna()
df = df.dropna()

(40761, 3)


In [ ]:
data_train = df.review.to_list()
label_train = df.label.to_list()
data_test = df_test.review.tolist()
label_test = df_test.label.tolist()

In [ ]:
!pip install keras-bert
!pip install keras-rectified-adam

In [ ]:
pip install tensorflow==2.2.0

In [ ]:
pip install keras==2.3.1

In [ ]:
import codecs
import tensorflow as tf
from tensorflow import keras
import os
from keras_radam import RAdam
from tensorflow.keras import backend as K
from keras_bert import load_trained_model_from_checkpoint
import numpy as np

In [ ]:
!wget -q https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip -o multi_cased_L-12_H-768_A-12.zip

In [ ]:
pretrained_path = '/content/drive/MyDrive/JK_Research/Sentiment-Analysis-using-BERT-master/BERT-embedding-CNN/multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [ ]:
SEQ_LEN = 128

In [ ]:
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    trainable=True,
    seq_len=SEQ_LEN,
    output_layer_num=4
)

In [ ]:
import keras_bert
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D
from tensorflow.keras.layers import Conv1D, MaxPool1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,LSTM,GRU

In [ ]:
LR=2e-5

In [ ]:
class NonMasking(keras.layers.Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [ ]:
inputs = model.inputs
bert_out = NonMasking()(model.outputs)

bert_out = SpatialDropout1D(0.5)(bert_out)

filter_lengths = [3, 4, 5]
conv_layers = []
for filter_length in filter_lengths:
    conv_layer = Conv1D(filters=100, kernel_size=filter_length, padding='valid',
                        strides=1, activation='relu')(bert_out)
    maxpooling = MaxPool1D(pool_size=SEQ_LEN - filter_length + 1)(conv_layer)
    flatten = Flatten()(maxpooling)
    conv_layers.append(flatten)
sentence_embed = Concatenate()(conv_layers)

dense_layer = Dense(128, activation='relu')(sentence_embed)
preds = Dense(1, activation='sigmoid')(dense_layer)
model1 = Model(inputs, preds)

In [ ]:
model1.compile(loss='binary_crossentropy',
              optimizer=RAdam(learning_rate=LR),
              metrics=['acc'])

In [ ]:
token_dict = {}
with codecs.open(vocab_path, 'rb','utf-8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [ ]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict,cased=True)

In [ ]:
from sklearn.model_selection import train_test_split
def load_data(data, sentiments):
    global tokenizer
    indices = []
    for text in data:
      ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
      indices.append(ids)

    return [indices, np.zeros_like(indices)], np.array(sentiments)

In [ ]:
SEQ_LENdata_train_,data_valid,label_train_,label_valid = train_test_split(data_train,label_train,test_size=0.15,random_state=48)

In [ ]:
X_train,Y_train = load_data(data_train_,label_train_)
X_valid,Y_valid = load_data(data_valid,label_valid)
X_test,Y_test = load_data(data_test,label_test)

In [ ]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [ ]:
from keras.callbacks import ModelCheckpoint 
checkpoint = ModelCheckpoint(path+'model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model1.load_weights(path+'model.h5')

In [ ]:
model1.fit(X_train,Y_train,epochs=10,batch_size=16,verbose = 1,validation_data=[X_valid,Y_valid],callbacks=callbacks_list)   

In [ ]:
y_pred = np.round(model1.predict(X_test))

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
print(precision_score(Y_test,y_pred))
print(recall_score(Y_test,y_pred))
print(f1_score(Y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import itertools
matplotlib.rcParams.update({'font.size': 16})
labels = [0,1]
cn = confusion_matrix(Y_test,y_pred,labels=labels)

In [ ]:
#@title plot confusion matrix
def plot_confusion_matrix(cm,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True,
                          target_names=None,
                          path_file='1.svg'):
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(path_file,format='svg')
    plt.show()
    

In [ ]:
plot_confusion_matrix(cn,title='Data vreview',normalize=False,target_names=labels,path_file=path+'C_M.svg')